In [9]:
import sqlite3
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [3]:
db_path = "../../db.sqlite3"  # Remplacez par le chemin vers votre base SQLite
conn = sqlite3.connect(db_path)

In [4]:
query = """
SELECT 
    orders.client_id,
    menu.name AS pizza_name
FROM 
    orders
JOIN 
    menu
ON 
    orders.pizza_id = menu.pizza_id
"""
data = pd.read_sql_query(query, conn)

In [7]:
# Calculer le nombre de commandes par client et pizza (fréquence comme score)
pizza_counts = data.groupby(['client_id', 'pizza_name']).size().reset_index(name='rating')

# Préparer les données pour Surprise
reader = Reader(rating_scale=(1, pizza_counts['rating'].max()))  # Échelle ajustée à la fréquence
data_surprise = Dataset.load_from_df(pizza_counts[['client_id', 'pizza_name', 'rating']], reader)

In [12]:
# Diviser les données en ensembles d'entraînement et de test
trainset, testset = train_test_split(data_surprise, test_size=0.2)

# Entraîner un modèle SVD
algo = SVD()
algo.fit(trainset)

In [31]:
# Faire des prédictions sur l'ensemble de test
predictions = algo.test(testset)

# Calculer les métriques
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")


RMSE: 0.8921
MAE:  0.7071
RMSE: 0.8920619420528594
MAE: 0.7071454750774768


In [14]:
from collections import defaultdict

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """
    Calculer la précision et le rappel à k pour une liste de prédictions.

    Args:
    predictions (list of Prediction): Liste des prédictions générées par Surprise.
    k (int): Nombre maximum de recommandations à considérer.
    threshold (float): Seuil au-dessus duquel une prédiction est considérée comme pertinente.

    Returns:
    dict: Dictionnaire contenant la précision et le rappel pour chaque utilisateur.
    """
    # Regrouper les prédictions par utilisateur
    user_est_true = defaultdict(list)
    for pred in predictions:
        user_est_true[pred.uid].append((pred.est, pred.r_ui))  # Note prédite (est), note réelle (r_ui)

    precisions = {}
    recalls = {}

    for uid, user_ratings in user_est_true.items():
        # Trier les prédictions de l'utilisateur par note prédite (descendante)
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Prendre les k premières prédictions
        user_ratings = user_ratings[:k]

        # Nombre de prédictions pertinentes
        n_relevant = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Nombre total d'items pertinents pour cet utilisateur
        n_total_relevant = sum((true_r >= threshold) for (_, true_r) in user_est_true[uid])

        # Calculer la précision et le rappel
        precisions[uid] = n_relevant / k if k > 0 else 0
        recalls[uid] = n_relevant / n_total_relevant if n_total_relevant > 0 else 0

    return precisions, recalls

In [15]:
# Calculer les prédictions sur l'ensemble de test
predictions = algo.test(testset)

# Calculer précision et rappel à k
k = 3  # Nombre de recommandations à considérer
threshold = 0.8  # Seuil pour considérer une recommandation pertinente
precisions, recalls = precision_recall_at_k(predictions, k=k, threshold=threshold)

# Calculer les moyennes globales
avg_precision = sum(precisions.values()) / len(precisions)
avg_recall = sum(recalls.values()) / len(recalls)

print(f"Précision moyenne à {k} : {avg_precision:.2f}")
print(f"Rappel moyen à {k} : {avg_recall:.2f}")

Précision moyenne à 3 : 0.51
Rappel moyen à 3 : 0.99


In [30]:
# Tester le modèle sur un client existant
def test_model_on_existing_client(client_id):
    # Vérifier si le client existe
    if client_id not in data['client_id'].unique():
        print(f"Erreur : client_id {client_id} n'existe pas dans les données.")
        return

    # Récupérer l'historique des commandes du client
    client_history = data[data['client_id'] == client_id].groupby('pizza_name').size()
    print(f"Historique des commandes pour le client {client_id} :")
    for pizza, count in client_history.items():
        print(f" - {pizza} (commandée {count} fois)")

    # Considérer toutes les pizzas, y compris celles déjà commandées
    pizza_names = data['pizza_name'].unique()

    # Générer des prédictions pour toutes les pizzas
    recommendations = [algo.predict(client_id, pizza_name) for pizza_name in pizza_names]

    # Trier les recommandations par score estimé
    top_recommendations = sorted(recommendations, key=lambda x: x.est, reverse=True)[:3]

    # Afficher les recommandations
    print(f"\nTop 3 recommandations pour le client {client_id} :")
    for rec in top_recommendations:
        print(f"Pizza : {rec.iid}, Score estimé : {rec.est}")

# Exemple : Tester sur un client existant
test_client_id = 13 # Remplacez par un client_id existant dans votre base
test_model_on_existing_client(test_client_id)

Historique des commandes pour le client 13 :
 - The Italian Capocollo Pizza (commandée 1 fois)
 - The Sicilian Pizza (commandée 1 fois)
 - The Spinach and Feta Pizza (commandée 5 fois)

Top 3 recommandations pour le client 13 :
Pizza : The Spinach and Feta Pizza, Score estimé : 2.6240186055693036
Pizza : The Hawaiian Pizza, Score estimé : 2.2508803674566242
Pizza : The Calabrese Pizza, Score estimé : 2.1466241138202924
